## Overview

The goal of this notebook is to gather data from the National Transit Database (NTD) and parse it into something usable for our transit carbon calculations. There are two datasets we need to import and parse, one being [NTD Annual Data - Fuel and Energy](https://data.transportation.gov/Public-Transit/2022-NTD-Annual-Data-Fuel-and-Energy/8ehq-7his/data) set, and the other being [NTD Service](https://www.transit.dot.gov/ntd/data-product/2022-service).

#### To Maintain

To update the data each year, update `urls` below with the most recent download links.

In [13]:
ntd_urls = {
  2022: {
    # https://www.transit.dot.gov/ntd/data-product/2022-fuel-and-energy
    "fuel_energy_csv": "https://data.transportation.gov/api/views/8ehq-7his/rows.csv?date=20231027&accessType=DOWNLOAD&bom=true&format=true",
    # https://www.transit.dot.gov/ntd/data-product/2022-service
    # Service by Mode csv
    "service_csv": "https://data.transportation.gov/api/views/4fir-qbim/rows.csv?date=20231102&accessType=DOWNLOAD&bom=true&format=true",
    # https://www.transit.dot.gov/ntd/data-product/2022-metrics not used
    # "metrics_csv": "https://data.transportation.gov/api/views/ekg5-frzt/rows.csv?date=20231102&accessType=DOWNLOAD&bom=true&format=true",
    # https://www.transit.dot.gov/ntd/data-product/2022-annual-database-fare-revenues
    "fare_revenue_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2024-04/2022%20Fare%20Revenue.xlsx",
    # https://www.transit.dot.gov/ntd/data-product/2022-annual-database-agency-information
    "agency_info_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2024-07/2022%20Agency%20Information_1-3_0.xlsx",
    # https://www.transit.dot.gov/ntd/data-product/2022-annual-database-service
    "service_times_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2024-04/2022%20Service.xlsx",
    # https://data.transportation.gov/resource/wwdp-t4re.json
    # "speed_api": "https://data.transportation.gov/resource/wwdp-t4re.json"
  },
  2021: {
    # https://www.transit.dot.gov/ntd/data-product/2021-fuel-and-energy
    "fuel_energy_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2023-12/2021%20Fuel%20and%20Energy_1-1_0.xlsx",
    # https://www.transit.dot.gov/ntd/data-product/2021-service
    "service_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2022-10/2021%20Service_static.xlsx",
    # https://www.transit.dot.gov/ntd/data-product/2021-metrics not used
    # "metrics_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2022-10/2021%20Metrics_static.xlsx",
    # https://www.transit.dot.gov/ntd/data-product/2021-annual-database-fare-revenues
    "fare_revenue_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2022-10/2021%20Fare%20Revenue.xlsx", 
    # https://www.transit.dot.gov/ntd/data-product/2021-annual-database-agency-information
    "agency_info_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2022-10/2021%20Agency%20Information.xlsx",
    # https://www.transit.dot.gov/ntd/data-product/2021-annual-database-service
    "service_times_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2022-10/2021%20Service.xlsx",
  },
  2020: {
    # https://www.transit.dot.gov/ntd/data-product/2020-fuel-and-energy
    "fuel_energy_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2023-12/2020-Fuel%20and%20Energy_1-1_1.xlsx",
    # https://www.transit.dot.gov/ntd/data-product/2020-service
    "service_zip": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2020-Service.zip",
    # https://www.transit.dot.gov/ntd/data-product/2020-metrics not used
    # "metrics_zip": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2020-Metrics.zip",
    # https://www.transit.dot.gov/ntd/data-product/2020-annual-database-fare-revenues,
    "fare_revenue_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2021-11/2020%20Fare%20Revenue.xlsx",
    # https://www.transit.dot.gov/ntd/data-product/2020-annual-database-agency-information
    "agency_info_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2021-11/2020%20Agency%20Information.xlsx",
    # https://www.transit.dot.gov/ntd/data-product/2020-annual-database-service
    "service_times_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2021-11/2020%20Service.xlsx",
  },
  2019: {
    # https://www.transit.dot.gov/ntd/data-product/2019-fuel-and-energy
    "fuel_energy_zip": "https://www.transit.dot.gov/sites/fta.dot.gov/files/Fuel%20and%20Energy.zip",
    # https://www.transit.dot.gov/ntd/data-product/2019-service
    "service_zip": "https://www.transit.dot.gov/sites/fta.dot.gov/files/Service.zip",
    # https://www.transit.dot.gov/ntd/data-product/2019-metrics, not used
    # "metrics_zip": "https://www.transit.dot.gov/sites/fta.dot.gov/files/Metrics.zip",
    # https://www.transit.dot.gov/ntd/data-product/2019-annual-database-fare-revenues
    "fare_revenue_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2020-10/2019%20Fare%20Revenue.xlsx",
    # https://www.transit.dot.gov/ntd/data-product/2019-annual-database-agency-information
    "agency_info_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2021-05/2019%20Agency%20Info.xlsx",
    # https://www.transit.dot.gov/ntd/data-product/2019-annual-database-service
    "service_times_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2020-10/2019%20Service.xlsx",
  },
  2018: {
    # https://www.transit.dot.gov/ntd/data-product/2018-fuel-and-energy
    "fuel_energy_xlsm": "https://www.transit.dot.gov/sites/fta.dot.gov/files/Fuel%20and%20Energy_3.xlsm",
    # https://www.transit.dot.gov/ntd/data-product/2018-service
    "service_xlsm": "https://www.transit.dot.gov/sites/fta.dot.gov/files/Service_4.xlsm",
    # https://www.transit.dot.gov/ntd/data-product/2018-metrics not used
    # "metrics_xlsm": "https://www.transit.dot.gov/sites/fta.dot.gov/files/Metrics_2.xlsm",
    # https://www.transit.dot.gov/ntd/data-product/2018-annual-database-fare-revenue
    "fare_revenue_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2018%20Fare%20Revenue.xlsx",
    # https://www.transit.dot.gov/ntd/data-product/2018-annual-database-agency-information
    "agency_info_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2018%20Agency%20Info_1.xlsx",
    # https://www.transit.dot.gov/ntd/data-product/2018-annual-database-service
    "service_times_xlsx": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2018%20Service.xlsx",
  }
}

# XLSX detailing urbanized areas; establishes relationship UACE Codes and UZA codes (which were used by NTD pre-2022)
census_changes_xlsx = "https://www.transit.dot.gov/sites/fta.dot.gov/files/2023-10/UZA_CHANGES_1990-2020_2_5.xlsx"

# CSV with relationship between ZCTA codes and UACE codes (from us_geo on GitHub)
zcta_urban_areas_urls = {
  2020: "https://raw.githubusercontent.com/bdurand/us_geo/master/data/dist/zcta_urban_areas.csv",
  2010: "https://raw.githubusercontent.com/bdurand/us_geo/master/data/2020_dist/zcta_urban_areas.csv"
}
# CSV with relationship between ZCTA codes and UZA codes (from acs-aggregate on GitHub)
zip_to_zcta_url = "https://raw.githubusercontent.com/censusreporter/acs-aggregate/master/crosswalks/zip_to_zcta/zip_zcta_xref.csv"

In [14]:
# imports

import pandas as pd
import matplotlib.pyplot as plt
import json
from pprint import pprint
from datetime import time
import datetime
import numpy as np
import os

from script_utils import is_up_to_date, load_dataframe

# move to the 'src' directory so we can import from the 'emcommon' package
%cd ../src
import emcommon.metrics.footprint.util as emcmfu

/Users/jfleisc1/Desktop/nrel/e-mission-common/src


In [15]:
# create UZA to UACE mapping using the NTD's census_changes_xlsx

census_changes_df = pd.read_excel(census_changes_xlsx)
census_changes_df = census_changes_df[["2010 NTD UZA Code (Deprecated)", "2020 UACE"]].dropna().astype(int)
# pad to 5 digits
census_changes_df['2020 UACE'] = census_changes_df['2020 UACE'].apply(lambda x: str(x).zfill(5))
uza_to_uace = census_changes_df.set_index("2010 NTD UZA Code (Deprecated)")["2020 UACE"].to_dict()
print("uza_to_uace: " + json.dumps(uza_to_uace))

uza_to_uace: {"1": "63217", "2": "51445", "3": "16264", "4": "56602", "7": "40429", "6": "22042", "5": "69076", "8": "92242", "9": "03817", "10": "09271", "12": "69184", "11": "23824", "14": "80389", "13": "78904", "15": "78661", "16": "57628", "17": "86599", "18": "23527", "22": "75340", "19": "04843", "23": "47995", "20": "77770", "24": "71317", "26": "78580", "28": "77068", "32": "65863", "21": "79093", "29": "79039", "37": "04384", "27": "69697", "25": "17668", "33": "41212", "30": "16885", "31": "43912", "36": "19234", "34": "90892", "38": "15670", "35": "57466", "39": "72505", "40": "42346", "42": "78499", "44": "61273", "50": "73261", "45": "74746", "41": "56116", "51": "65080", "47": "37243", "43": "51755", "49": "62677", "46": "11350", "48": "10162", "52": "88732", "53": "27253", "54": "39889", "58": "65269", "57": "52390", "64": "09536", "55": "07786", "56": "01171", "62": "88948", "63": "31843", "60": "75664", "76": "15508", "59": "22528", "69": "57709", "73": "18856", "68":

In [16]:
ntd_fuel_types = {
    "Gasoline": {"given_unit": "gal", "kwh_per_unit": emcmfu.KWH_PER_GAL_GASOLINE, "alias": "Gasoline"},
    "Diesel": {"given_unit": "gal", "kwh_per_unit": emcmfu.KWH_PER_GAL_DIESEL, "alias": "Diesel"},
    "Bio-Diesel": {"given_unit": "gal", "kwh_per_unit": emcmfu.KWH_PER_GAL_BIODIESEL, "alias": "Diesel"},
    "Liquefied Petroleum Gas": {"given_unit": "gal equivalent", "kwh_per_unit": emcmfu.KWH_PER_GAL_LPG, "alias": "LPG"},
    "Compressed Natural Gas": {"given_unit": "gal equivalent", "kwh_per_unit": emcmfu.KWH_PER_GAL_CNG, "alias": "CNG"},
    "Hydrogen": {"given_unit": "kg", "kwh_per_unit": emcmfu.KWH_PER_KG_HYDROGEN, "alias": "Hydrogen"},
    "Electric Propulsion": {"given_unit": "kWh", "kwh_per_unit": 1, "alias": "Electric"},
    "Electric Battery": {"given_unit": "kWh", "kwh_per_unit": 1, "alias": "Electric"},
    "Other Fuel": {"given_unit": "gal/gal equivalent", "kwh_per_unit": emcmfu.KWH_PER_GAL_OTHER, "alias": "Other"}
}
fuel_types = ['Gasoline', 'Diesel', 'LPG', 'CNG', 'Hydrogen', 'Electric', 'Other']
fuel_types_all = fuel_types + ['All Fuels']

def agg_by_ntd_id_and_mode(df):
    # Remove commas, convert to numeric, and sort
    df = df.replace({',': ''}, regex=True)
    df = df.apply(pd.to_numeric, errors='ignore')
    df = df.sort_values(by=['NTD ID', 'Mode'])

    # Define the columns that should not be summed
    possible_cols = ['City', 'State', 'NTD ID', 'Mode', 'Mode Name', 'Organization Type', 'Reporter Type', 'Primary UZA Population', 'Agency']
    non_sum_columns = [col for col in possible_cols if col in df.columns]

    # Define the aggregation functions for each column
    agg_funcs = {col: 'first' for col in non_sum_columns}
    for col in df.columns:
        if col not in non_sum_columns:
            agg_funcs[col] = 'sum'

    # Group by 'NTD ID' and 'Mode' and apply the aggregation functions
    df = df.groupby(['NTD ID', 'Mode'], as_index=False).agg(agg_funcs)

    return df

In [17]:
def make_fuel_energy_df(y):
    df = load_dataframe(ntd_urls[y], 'fuel_energy', 'Fuel and Energy')
    col_renames = {
        # kwh -> kWh
        'Electric Propulsion (kwh)': 'Electric Propulsion (kWh)',
        'Electric Battery (kwh)': 'Electric Battery (kWh)',
        # require renames as older years did not include (miles) in the column name
        'Diesel': 'Diesel (miles)',
        'Gasoline': 'Gasoline (miles)',
        'Liquefied Petroleum Gas': 'Liquefied Petroleum Gas (miles)',
        'Compressed Natural Gas': 'Compressed Natural Gas (miles)',
        # 'Bio-Diesel': 'Bio-Diesel (miles)',
        'Hydrogen': 'Hydrogen (miles)',
        'Other Fuel': 'Other Fuel (miles)',
        'Electric Propulsion': 'Electric Propulsion (miles)',
        'Electric Battery': 'Electric Battery (miles)',
    }
    df = df.rename(columns=col_renames)

    if 'UACE Code' not in df.columns:
        agency_info_df = load_dataframe(ntd_urls[y], 'agency_info')[['NTD ID', 'Primary UZA']]
        agency_info_df['UACE Code'] = agency_info_df['Primary UZA'].map(uza_to_uace)
        ntd_id_uace_map = agency_info_df.set_index('NTD ID')['UACE Code'].to_dict()
    else:
        ntd_id_uace_map = df.set_index('NTD ID')['UACE Code'].to_dict()


    df = agg_by_ntd_id_and_mode(df)

    df['UACE Code'] = df['NTD ID'].map(ntd_id_uace_map)
    print(f"dropping {len(df[df['UACE Code'].isna()])} rows with missing UACE Code")
    df = df.dropna(subset=['UACE Code'])
    df['UACE Code'] = df['UACE Code'].apply(lambda x: str(x).zfill(5))

    # for all fuel types, convert to km
    for fuel in ntd_fuel_types:
        old_col_name = fuel + ' (miles)'
        new_col_name = ntd_fuel_types[fuel]['alias'] + ' (km)'
        if new_col_name in df.columns and old_col_name in df.columns:
            df[new_col_name] += df[old_col_name] / emcmfu.MI_PER_KM
        elif old_col_name in df.columns:
            df[new_col_name] = df[old_col_name] / emcmfu.MI_PER_KM
        elif new_col_name not in df.columns:
            df[new_col_name] = 0

 

    # for all fuel types, convert to kWh
    def get_old_col_name(fuel_type): return fuel_type + ' (' + ntd_fuel_types[fuel_type]['given_unit'] + ')'
    for fuel in ntd_fuel_types:
        old_col_name = get_old_col_name(fuel)
        new_col_name = ntd_fuel_types[fuel]['alias'] + ' (kWh)'

        if new_col_name in df.columns and old_col_name in df.columns:
            df[new_col_name] += df[old_col_name] * ntd_fuel_types[fuel]['kwh_per_unit']
        elif old_col_name in df.columns:
            df[new_col_name] = df[old_col_name] * ntd_fuel_types[fuel]['kwh_per_unit']
        elif new_col_name not in df.columns:
            df[new_col_name] = 0

    # any rows where km is present but kWh is not, or vice versa, fill with 0
    for fuel in fuel_types:
        df[df[fuel + ' (km)'].notna()].where(df[fuel + ' (kWh)'].isna(), 0, inplace=True)
        df[df[fuel + ' (kWh)'].notna()].where(df[fuel + ' (km)'].isna(), 0, inplace=True)

    df['All Fuels (kWh)'] = sum([df[f + ' (kWh)'].fillna(0) for f in fuel_types])
    df['All Fuels (km)'] = sum([df[f + ' (km)'].fillna(0) for f in fuel_types])
    df = df[df['All Fuels (km)'] != 0]
    for fuel in fuel_types_all:
        df[f"{fuel} (%)"] = df[f"{fuel} (km)"] / df['All Fuels (km)'] * 100

    # now add a wh/km column for each fuel type
    for fuel in fuel_types_all:
        if f"{fuel} (km)" not in df.columns: continue
        df[f"{fuel} (Wh/km)"] = (df[f"{fuel} (kWh)"] * 1000 / df[f"{fuel} (km)"])
        # fill missing values with 0
        df[f"{fuel} (Wh/km)"] = df[f"{fuel} (Wh/km)"].fillna(0)
        # replace infinite values with 0
        df[f"{fuel} (Wh/km)"] = df[f"{fuel} (Wh/km)"].replace([float('inf'), float('-inf')], 0)

  
    return df

In [18]:
def make_service_df(y):
    df = load_dataframe(ntd_urls[y], 'service', 'Annual Service Data By Mode')
    # if 'Actual Vehicles Passenger Car Miles' in df.columns:
        # print(f"Year {y} has 'Actual Vehicles Passenger Car Miles' column")
 
    print('!'*90)
    print(df.columns)
    col_renames = {
        'Passenger Miles Traveled': 'Passenger Miles',
        'Unlinked Passenger Trips (UPT)': 'Unlinked Passenger Trips',
        'Actual Vehicle/Passenger Car Miles': 'Vehicle Miles',
        'Actual Vehicles Passenger Car Miles': 'Vehicle Miles',
        'sum_actual_vehicles_passenger_car_miles': 'Vehicle Miles',
        'sum_train_miles': 'Train Miles',
        'sum_unlinked_passenger_trips_upt': 'Unlinked Passenger Trips',
        'sum_passenger_miles': 'Passenger Miles',
    }
    df = df.rename(columns=col_renames)
    df = agg_by_ntd_id_and_mode(df)

    # convert miles columns to km
    miles_cols = ['Vehicle Miles', 'Train Miles', 'Passenger Miles']
    for col in miles_cols:
        new_col_name = col.replace('Miles', 'km')
        df[new_col_name] = df[col] / emcmfu.MI_PER_KM
    df = df.drop(columns=miles_cols)

    # remove rows that have "Passenger km" = 0 or "Vehicle km" = 0
    df = df[(df['Passenger km'] != 0) & (df['Vehicle km'] != 0)]

    # For "Average Passengers", we want the passenger load on the engine
    # For rail modes, this is 'Train km'
    # For non-rail modes, this is 'Vehicle km' ('Train km' will be 0)
    # The average number of passengers is 'Passenger km' / ('Train km' || 'Vehicle km')
    engine_km = df['Train km'].replace(0, pd.NA).fillna(df['Vehicle km'])
    df['Average Passengers'] = df['Passenger km'] / engine_km

    return df

In [19]:
def make_fares_df(y):
    df = load_dataframe(ntd_urls[y], 'fare_revenue')

    df = agg_by_ntd_id_and_mode(df)

    # we can only find average fare after merging with upt

    return df

In [20]:
def make_headway_dict(y):
    headway_df = load_dataframe(ntd_urls[y], 'service_times')
    # display(headway_df['Time Service Begins'].head(55))
 
    # Headway logic
    # for each unique NTD ID, there are rows with particular Mode. more than one Mode can have the same NTD ID.
    # for each Mode, there can be rows with one TOS and rows with another TOS.
    # add the Average Weekday - AM Peak, Average Weekday - Midday, Average Weekday - PM Peak,
    # Average Typical Weekday, Average Typical Saturday, Average Typical Sunday values. those are the
    # Time Period possible columns, but we are interested in the 'Time Service Begins' and
    # 'Time Service Ends' columns belonging to those particular Time Periods
    non_rail_modes = {'DR', 'CB', 'MB', 'RB', 'VP', 'FB', 'TB', 'PB', 'TR', 'AR'}


    headway = {}
    for ntd_id, group in headway_df.groupby('NTD ID'):
        headway[ntd_id] = {}
        for mode, mode_group in group.groupby('Mode'):
            headway[ntd_id][mode] = {}
            
            for tos, tos_group in mode_group.groupby('TOS'):
                headway[ntd_id][mode][tos] = {}
                for time_period, time_period_group in tos_group.groupby('Time Period'):
                    
                    if mode in non_rail_modes:
                        if 'Actual Vehicles/Passenger Car Revenue Miles' in time_period_group.columns:
                            columnname = 'Actual Vehicles/Passenger Car Revenue Miles'
                        elif 'Actual Vehicle/ Passenger Car Revenue Miles' in time_period_group.columns:
                            columnname = 'Actual Vehicle/ Passenger Car Revenue Miles'
                        elif 'Actual Vehicle/Passenger Car Revenue Miles' in time_period_group.columns:
                            columnname = 'Actual Vehicle/Passenger Car Revenue Miles'
                        elif 'Actual Vehicles/ Passenger Car Revenue Miles' in time_period_group.columns:
                            columnname = 'Actual Vehicles/ Passenger Car Revenue Miles'
                        else:
                            print(time_period_group.columns)
                            raise KeyError
                        try:
                            revenue_miles = time_period_group[columnname].dropna()
                        except KeyError:
                            print(time_period_group.columns)
                            raise KeyError
                        hour_col_name = 'Actual Vehicle/Passenger Car Revenue Hours' if \
                            'Actual Vehicle/Passenger Car Revenue Hours' in time_period_group.columns else 'Actual Vehicle/ Passenger Car Revenue Hours'
                        # try:
                        revenue_hours = time_period_group[hour_col_name].dropna()
                        # except KeyError:
                            # print(time_period_group.columns)
                            # raise KeyError
                    else:
                        revenue_miles = time_period_group['Train Revenue Miles'].dropna()
                        revenue_hours = time_period_group['Train Revenue Hours'].dropna()
                    
                    # Most agencies do not give revenue miles and hours
                    # for more specific time periods such as the Peak
                    if len(revenue_miles) == 1 and len(revenue_hours) == 1:
                        revenue_miles = revenue_miles.iloc[0]
                        revenue_hours = revenue_hours.iloc[0]
                        if revenue_hours > 0:
                            average_speed = revenue_miles / revenue_hours
                        else:
                            average_speed = None
                    else:
                        average_speed = None
                    
                    time_service_begins = time_period_group['Time Service Begins'].dropna()
                    time_service_ends = time_period_group['Time Service Ends'].dropna()

                    if not time_service_begins.empty and not time_service_ends.empty:
                        headway[ntd_id][mode][tos][time_period] = {
                            'Time Service Begins': min(time_service_begins),
                            'Time Service Ends': max(time_service_ends),
                            'Average Speed': average_speed,
                            'Revenue Miles': revenue_miles if not isinstance(revenue_miles, pd.Series) else None,
                            'Revenue Hours': revenue_hours if not isinstance(revenue_hours, pd.Series) else None,
                        }
                    else:
                        headway[ntd_id][mode][tos][time_period] = {
                            'Time Service Begins': None,
                            'Time Service Ends': None,
                            'Average Speed': average_speed,
                            'Revenue Miles': revenue_miles if not isinstance(revenue_miles, pd.Series) else None,
                            'Revenue Hours': revenue_hours if not isinstance(revenue_hours, pd.Series) else None,
                        }

    # Find the earliest and latest TOS for each NTD ID and Mode
    # pedestrians do not care to take directly operated or not.
    # they just want to get home with whats available
    # however, we must keep them distinct because different revenue miles and speed etc

    #unused
    earliest_latest_headway = {}
    for ntd_id, mode_dict in headway.items():
        earliest_latest_headway[ntd_id] = {}
        for mode, tos_dict in mode_dict.items():
            earliest_latest_headway[ntd_id][mode] = {
                'Average Typical Weekday': {},
                'Average Typical Saturday': {},
                'Average Typical Sunday': {}
            }
            for key in ['Average Typical Weekday', 'Average Typical Saturday', 'Average Typical Sunday']:
                for tos, time_period_dict in tos_dict.items():
                    if isinstance(time_period_dict, dict):  # Ensure time_period_dict is a dictionary
                        for time_period, times in time_period_dict.items():
                            if time_period == key:
                                earliest_latest_headway[ntd_id][mode][key][tos] = {
                                    'Time Service Begins': times['Time Service Begins'],
                                    'Time Service Ends': times['Time Service Ends']
                                }

    # Helper function to convert datetime.time objects to strings
    def convert_time_to_string(data):
        if isinstance(data, dict):
            return {k: convert_time_to_string(v) for k, v in data.items()}
        elif isinstance(data, list):
            return [convert_time_to_string(i) for i in data]
        elif isinstance(data, datetime.time):
            return data.strftime('%H:%M:%S')
        elif isinstance(data, np.float64):
            return float(data)
        else:
            return data

    # Convert datetime.time objects to strings in the headway dictionary
    headway = convert_time_to_string(headway)

    # BEGIN: Print first five key-value pairings from headway
    for ntd_id, modes in list(headway.items())[:6]:
        print(ntd_id)
        pprint(modes)
    # END:
    
    return headway

In [21]:
# merge the dataframes
def make_merged_ntd_df(y):
    fuel_energy_df = make_fuel_energy_df(y)
    service_df = make_service_df(y)
    fares_df = make_fares_df(y)

    
    df = fuel_energy_df.merge(service_df, on=['NTD ID', 'Mode'], suffixes=('', '_service'))

    df = df.merge(fares_df, on=['NTD ID', 'Mode'], suffixes=('', '_fare'))

    # headway_dict = make_headway_dict(y)
    # df['Headway Dict'] = df['NTD ID'].map(headway_dict)
    
    # display(df)
    # print('^' * 80)
    # import time as hey; hey.sleep(99999)
    # calculate average fare
    
    df['Average Fare'] = df['Passenger Paid Fares'] / df['Unlinked Passenger Trips']

    # convert (Wh/km) columns to (Wh/pkm) columns
    for f in fuel_types_all:
        df[f"{f} (Wh/pkm)"] = df[f"{f} (Wh/km)"] / df['Average Passengers']

    # only keep relevant columns
    df = df[['NTD ID', 'Mode', 'UACE Code']
            + [f + ' (%)' for f in fuel_types_all]
            + [f + ' (Wh/pkm)' for f in fuel_types_all]
            + ['Unlinked Passenger Trips', 'Passenger Paid Fares', 'Average Fare', 'Average Passengers']
            # + ['Headway Dict']
            ]

    return df


In [22]:

# Function to generate a unique filename
def save_figure(title="plot", output_dir="plots", file_format="pdf"):
    title = title.replace(" ", "_").replace('/', '_').lower()
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    i = 1
    while True:
        filename = f"{title}_{i}.{file_format}"
        file_path = os.path.join(output_dir, filename)
        if not os.path.exists(file_path):
            plt.savefig(file_path, format=file_format)
            print(f"Saved figure as {file_path}")
            break
        i += 1
    plt.close()  # Close the figure after saving to avoid display issues

In [23]:

def plot_whpkm_hists(df, title, year):
    modes = df['Mode'].unique()
    _, axs = plt.subplots(1, 2, figsize=(15, 5))
    
    # Left plot: Distribution by Mode
    for mode in modes:
        values = []
        for fuel_type in fuel_types_all:
            values.extend(df[df['Mode'] == mode][fuel_type + ' (Wh/pkm)'].dropna())
        axs[0].hist(values, bins=50, alpha=0.5, label=mode, log=True)
        print(f"{mode} desc: {pd.Series(values).astype(float).describe()}")

    axs[0].set_xlabel('Wh/pkm')
    axs[0].set_ylabel('Frequency')
    axs[0].set_title(f'{title} for {year}, by Mode')
    axs[0].legend(title='Mode')

    # Right plot: Distribution by Fuel Type
    for fuel_type in fuel_types_all:
        axs[1].hist(df[fuel_type + ' (Wh/pkm)'].dropna(), bins=50, alpha=0.5, label=fuel_type, log=True)
        print(f"{fuel_type} desc: {df[fuel_type + ' (Wh/pkm)'].dropna().astype(float).describe()}")
    
    axs[1].set_xlabel('Wh/pkm')
    axs[1].set_ylabel('Frequency')
    axs[1].set_title(f'{title} for {year}, by Fuel Type')
    axs[1].legend(title='Fuel Type')
    
    save_figure(f"{title}_{year}")



def remove_outliers(df, year):
    print("Removing outliers")
    modes = df['Mode'].unique()
    bounds = {}
    for mode in modes:
        q1 = df[df['Mode'] == mode]['All Fuels (Wh/pkm)'].quantile(0.25)
        q3 = df[df['Mode'] == mode]['All Fuels (Wh/pkm)'].quantile(0.75)
        iqr = q3 - q1
        bounds[mode] = (q1 - 1.5 * iqr, q3 + 1.5 * iqr)

    df = df.replace(0, pd.NA)
    for fuel_type in fuel_types_all:
        outliers = df[(df[fuel_type + ' (Wh/pkm)'] < df['Mode'].apply(lambda x: bounds[x][0])) | \
                      (df[fuel_type + ' (Wh/pkm)'] > df['Mode'].apply(lambda x: bounds[x][1]))].index
        for outlier in outliers:
            o = df.loc[outlier]
            print(f"Removing {o['Mode']} outlier: expected {bounds[o['Mode']]}, got {o.to_dict()}")
            df.at[outlier, fuel_type + ' (Wh/pkm)'] = pd.NA
            df.at[outlier, fuel_type + ' (%)'] = pd.NA
    
    plot_whpkm_hists(df, 'Distribution of Wh/pkm after removing outliers', year)

    return df


Make the dfs for each year and ouput `.json` files in `emcommon/metrics/footprint/ntd_data/`

In [24]:
for year in ntd_urls:
    urls = ntd_urls[year].values()
    output_filename = f"../src/emcommon/resources/ntd{year}_intensities.json"

    # if is_up_to_date(output_filename, urls):
      # continue

    df = make_merged_ntd_df(year)
    # display(df)
    # print('~'*90)
    df = remove_outliers(df, year)

    # convert to JSON
    df = df.convert_dtypes()
    records = df.to_dict(orient='records')
    # remove NA values since they are not JSON serializable and just take up space
    records = [ { k: v for k, v in record.items() if v and v == v } for record in records ]

    output = {
       "records": records,
       "metadata": {
          "year": year,
          "data_source_urls": list(urls)
      }
    }
    with open(output_filename, 'w') as f:
        json.dump(output, f, indent=2)

print("Done generating NTD intensities json files")

/var/folders/zp/zk66zxkj0_5d59vffwm9xkf5yvy8rv/T/ipykernel_1550/3991290417.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


dropping 0 rows with missing UACE Code
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Index(['NTD ID', 'max_agency', 'max_reporter_type', 'max_organization_type',
       'Report Year', 'max_city', 'max_state', 'max_agency_voms', 'Mode',
       'max_mode_name', 'Type Of Service', 'max_mode_voms',
       'max_primary_uza_code', 'max_primary_uza_name',
       'max_primary_uza_area_sq_miles', 'max_primary_uza_population',
       'max_service_area_sq_miles', 'max_service_area_population',
       'questionable_record', 'max_time_period', 'min_time_service_begins',
       'max_time_service_ends', 'sum_actual_vehicles_passenger_car_miles',
       'sum_actual_vehicles_passenger_car_revenue_miles',
       'sum_actual_vehicles_passenger_deadhead_miles',
       'sum_scheduled_vehicles_passenger_car_revenue_miles',
       'sum_actual_vehicles_passenger_car_hours',
       'sum_actual_vehicles_passenger_car_revenue_hours',
       'sum_actual_vehicles_passen

/var/folders/zp/zk66zxkj0_5d59vffwm9xkf5yvy8rv/T/ipykernel_1550/3991290417.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
/var/folders/zp/zk66zxkj0_5d59vffwm9xkf5yvy8rv/T/ipykernel_1550/146710289.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  engine_km = df['Train km'].replace(0, pd.NA).fillna(df['Vehicle km'])
/var/folders/zp/zk66zxkj0_5d59vffwm9xkf5yvy8rv/T/ipykernel_1550/3991290417.py:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')
D

Removing outliers
Removing DR outlier: expected (np.float64(-1468.0716538711886), np.float64(12758.181908996215)), got {'NTD ID': 16, 'Mode': 'DR', 'UACE Code': '51283', 'Gasoline (%)': 33.073059097843924, 'Diesel (%)': None, 'LPG (%)': 66.92694090215609, 'CNG (%)': None, 'Hydrogen (%)': None, 'Electric (%)': None, 'Other (%)': None, 'All Fuels (%)': 100.0, 'Gasoline (Wh/pkm)': 14526.566816514662, 'Diesel (Wh/pkm)': None, 'LPG (Wh/pkm)': 10028.823832256909, 'CNG (Wh/pkm)': None, 'Hydrogen (Wh/pkm)': None, 'Electric (Wh/pkm)': None, 'Other (Wh/pkm)': None, 'All Fuels (Wh/pkm)': 11516.365027509604, 'Unlinked Passenger Trips': 62271, 'Passenger Paid Fares': 17570.0, 'Average Fare': 0.2821538115655763, 'Average Passengers': 0.6866438536456962}
Removing DR outlier: expected (np.float64(-1468.0716538711886), np.float64(12758.181908996215)), got {'NTD ID': 10001, 'Mode': 'DR', 'UACE Code': '72505', 'Gasoline (%)': 2.5641052999974394, 'Diesel (%)': 97.43589470000256, 'LPG (%)': None, 'CNG (%)'

DEBUG:matplotlib.ticker:vmin 0.8184306128925417 vmax 67.20178733981619
DEBUG:matplotlib.ticker:ticklocs array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])
DEBUG:matplotlib.ticker:vmin 0.8184306128925417 vmax 67.20178733981619
DEBUG:matplotlib.ticker:ticklocs array([2.e-02, 3.e-02, 4.e-02, 5.e-02, 6.e-02, 7.e-02, 8.e-02, 9.e-02,
       2.e-01, 3.e-01, 4.e-01, 5.e-01, 6.e-01, 7.e-01, 8.e-01, 9.e-01,
       2.e+00, 3.e+00, 4.e+00, 5.e+00, 6.e+00, 7.e+00, 8.e+00, 9.e+00,
       2.e+01, 3.e+01, 4.e+01, 5.e+01, 6.e+01, 7.e+01, 8.e+01, 9.e+01,
       2.e+02, 3.e+02, 4.e+02, 5.e+02, 6.e+02, 7.e+02, 8.e+02, 9.e+02,
       2.e+03, 3.e+03, 4.e+03, 5.e+03, 6.e+03, 7.e+03, 8.e+03, 9.e+03])
DEBUG:matplotlib.ticker:vmin 0.8184306128925417 vmax 67.20178733981619
DEBUG:matplotlib.ticker:ticklocs array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])
DEBUG:matplotlib.ticker:vmin 0.8184306128925417 vmax 67.20178733981619
DEBUG:matplotlib.ticker:ticklocs array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02

...

Removing outliers
Removing DR outlier: expected (np.float64(-89.22410290098333), np.float64(5670.8027255276775)), got {'NTD ID': 6.0, 'Mode': 'DR', 'UACE Code': '97507', 'Gasoline (%)': 100.0, 'Diesel (%)': None, 'LPG (%)': None, 'CNG (%)': None, 'Hydrogen (%)': None, 'Electric (%)': None, 'Other (%)': None, 'All Fuels (%)': 100.0, 'Gasoline (Wh/pkm)': 11113.067118725423, 'Diesel (Wh/pkm)': None, 'LPG (Wh/pkm)': None, 'CNG (Wh/pkm)': None, 'Hydrogen (Wh/pkm)': None, 'Electric (Wh/pkm)': None, 'Other (Wh/pkm)': None, 'All Fuels (Wh/pkm)': 11113.067118725423, 'Unlinked Passenger Trips': 69112, 'Passenger Paid Fares': 83154, 'Average Fare': 1.2031774510938766, 'Average Passengers': 0.6205514274540478}
Removing DR outlier: expected (np.float64(-89.22410290098333), np.float64(5670.8027255276775)), got {'NTD ID': 25.0, 'Mode': 'DR', 'UACE Code': '78229', 'Gasoline (%)': 78.1206090549862, 'Diesel (%)': 21.879390945013803, 'LPG (%)': None, 'CNG (%)': None, 'Hydrogen (%)': None, 'Electric (%)':

DEBUG:matplotlib.ticker:vmin 0.8240203518378003 vmax 58.25098845307195
DEBUG:matplotlib.ticker:ticklocs array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])
DEBUG:matplotlib.ticker:vmin 0.8240203518378003 vmax 58.25098845307195
DEBUG:matplotlib.ticker:ticklocs array([2.e-02, 3.e-02, 4.e-02, 5.e-02, 6.e-02, 7.e-02, 8.e-02, 9.e-02,
       2.e-01, 3.e-01, 4.e-01, 5.e-01, 6.e-01, 7.e-01, 8.e-01, 9.e-01,
       2.e+00, 3.e+00, 4.e+00, 5.e+00, 6.e+00, 7.e+00, 8.e+00, 9.e+00,
       2.e+01, 3.e+01, 4.e+01, 5.e+01, 6.e+01, 7.e+01, 8.e+01, 9.e+01,
       2.e+02, 3.e+02, 4.e+02, 5.e+02, 6.e+02, 7.e+02, 8.e+02, 9.e+02,
       2.e+03, 3.e+03, 4.e+03, 5.e+03, 6.e+03, 7.e+03, 8.e+03, 9.e+03])
DEBUG:matplotlib.ticker:vmin 0.8240203518378003 vmax 58.25098845307195
DEBUG:matplotlib.ticker:ticklocs array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])
DEBUG:matplotlib.ticker:vmin 0.8240203518378003 vmax 58.25098845307195
DEBUG:matplotlib.ticker:ticklocs array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02

CR desc: count      50.000000
mean      333.387885
std       285.150534
min        12.056034
25%        76.944251
50%       282.772710
75%       462.513884
max      1024.176962
dtype: float64
AR desc: count      2.000000
mean     110.463809
std        0.000000
min      110.463809
25%      110.463809
50%      110.463809
75%      110.463809
max      110.463809
dtype: float64
TR desc: count      2.000000
mean     176.243363
std        0.000000
min      176.243363
25%      176.243363
50%      176.243363
75%      176.243363
max      176.243363
dtype: float64
HR desc: count    26.000000
mean     41.230723
std      23.485197
min      11.568722
25%      22.551414
50%      37.843691
75%      50.448267
max      89.070422
dtype: float64
IP desc: count       6.000000
mean     2871.250001
std      2931.439465
min       134.644684
25%       594.914790
50%      1975.725110
75%      5371.466435
max      6503.380210
dtype: float64
CC desc: count      2.000000
mean     306.104804
std        0.000000
min

DEBUG:matplotlib.ticker:vmin 0.8008091997686577 vmax 106.14263675361777
DEBUG:matplotlib.ticker:ticklocs array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04])
DEBUG:matplotlib.ticker:vmin 0.8008091997686577 vmax 106.14263675361777
DEBUG:matplotlib.ticker:ticklocs array([2.e-02, 3.e-02, 4.e-02, 5.e-02, 6.e-02, 7.e-02, 8.e-02, 9.e-02,
       2.e-01, 3.e-01, 4.e-01, 5.e-01, 6.e-01, 7.e-01, 8.e-01, 9.e-01,
       2.e+00, 3.e+00, 4.e+00, 5.e+00, 6.e+00, 7.e+00, 8.e+00, 9.e+00,
       2.e+01, 3.e+01, 4.e+01, 5.e+01, 6.e+01, 7.e+01, 8.e+01, 9.e+01,
       2.e+02, 3.e+02, 4.e+02, 5.e+02, 6.e+02, 7.e+02, 8.e+02, 9.e+02,
       2.e+03, 3.e+03, 4.e+03, 5.e+03, 6.e+03, 7.e+03, 8.e+03, 9.e+03,
       2.e+04, 3.e+04, 4.e+04, 5.e+04, 6.e+04, 7.e+04, 8.e+04, 9.e+04])
DEBUG:matplotlib.ticker:vmin 0.8008091997686577 vmax 106.14263675361777
DEBUG:matplotlib.ticker:ticklocs array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04])
DEBUG:matplotlib.ticker:vmin 0.8008091997686577 vmax 106.14

Saved figure as plots/distribution_of_wh_pkm_after_removing_outliers_2018_3.pdf
Done generating NTD intensities json files


# Test uace->fare

In [25]:
import emcommon.metrics.footprint.transit as emcmft



In [26]:
# metadata = {}

In [27]:
intensities = await emcmft.get_transit_intensities_for_uace(2022, '80389', ['FB'])


In [28]:
from pprint import pprint; pprint(intensities)


({'average_fare': 2.213850868836879,
  'diesel': {'weight': 1.0, 'wh_per_km': 5422.9242966123265},
  'overall': {'weight': 1.0, 'wh_per_km': 5422.9242966123265}},
 {'data_source_urls': ['https://data.transportation.gov/api/views/8ehq-7his/rows.csv?date=20231027&accessType=DOWNLOAD&bom=true&format=true',
                       'https://data.transportation.gov/api/views/4fir-qbim/rows.csv?date=20231102&accessType=DOWNLOAD&bom=true&format=true',
                       'https://www.transit.dot.gov/sites/fta.dot.gov/files/2024-04/2022%20Fare%20Revenue.xlsx',
                       'https://www.transit.dot.gov/sites/fta.dot.gov/files/2024-07/2022%20Agency%20Information_1-3_0.xlsx',
                       'https://www.transit.dot.gov/sites/fta.dot.gov/files/2024-04/2022%20Service.xlsx'],
  'data_sources': ['ntd2022'],
  'is_provisional': False,
  'ntd_ids': [1, 28, 35],
  'ntd_modes': ['FB'],
  'ntd_uace_code': '80389',
  'requested_year': 2022})


In [29]:
intensities = await emcmft.get_transit_intensities(2022, [-82.328132, 29.626142], modes=['MB'])


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): geocoding.geo.census.gov:443


https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x=-82.328132&y=29.626142&benchmark=Public_AR_Current&vintage=Census2020_Current&layers=87&format=json
####################################################################################################


DEBUG:urllib3.connectionpool:https://geocoding.geo.census.gov:443 "GET /geocoder/geographies/coordinates?x=-82.328132&y=29.626142&benchmark=Public_AR_Current&vintage=Census2020_Current&layers=87&format=json HTTP/11" 200 4954


In [30]:
pprint(intensities)

({'average_fare': 0.03361652051342716,
  'diesel': {'weight': 0.939763867305139, 'wh_per_km': 2731.3836289022956},
  'electric': {'weight': 0.01323088493189888, 'wh_per_km': 455.9238753207569},
  'gasoline': {'weight': 0.04700524776296217, 'wh_per_km': 1482.1310654698204},
  'overall': {'weight': 1.0, 'wh_per_km': 2642.555856472832}},
 {'data_source_urls': ['https://data.transportation.gov/api/views/8ehq-7his/rows.csv?date=20231027&accessType=DOWNLOAD&bom=true&format=true',
                       'https://data.transportation.gov/api/views/4fir-qbim/rows.csv?date=20231102&accessType=DOWNLOAD&bom=true&format=true',
                       'https://www.transit.dot.gov/sites/fta.dot.gov/files/2024-04/2022%20Fare%20Revenue.xlsx',
                       'https://www.transit.dot.gov/sites/fta.dot.gov/files/2024-07/2022%20Agency%20Information_1-3_0.xlsx',
                       'https://www.transit.dot.gov/sites/fta.dot.gov/files/2024-04/2022%20Service.xlsx'],
  'data_sources': ['ntd2022'],
  '